In [2]:
from importlib import reload
from openforbc.gpu import GPU, VGPUType
import openforbc.gpu; reload(openforbc.gpu)

gpus = GPU.get_gpus()
print([x.supported_vgpu_types for x in gpus])
print([x.get_creatable_vgpus() for x in gpus])
print([x.get_current_mig_status() for x in gpus])
print([x.get_vgpu_mode() for x in gpus])
print([x.get_vf_num() for x in gpus])

[[474: GRID A100-1-5C (MIG), 475: GRID A100-2-10C (MIG), 476: GRID A100-3-20C (MIG), 477: GRID A100-4-20C (MIG), 478: GRID A100-7-40C (MIG), 468: GRID A100-4C, 469: GRID A100-5C, 470: GRID A100-8C, 471: GRID A100-10C, 472: GRID A100-20C, 473: GRID A100-40C, 706: GRID A100-1-5CME (MIG)]]
[[476: GRID A100-3-20C (MIG), 474: GRID A100-1-5C (MIG), 706: GRID A100-1-5CME (MIG)]]
[<MIGModeStatus.ENABLE: 1>]
[<VGPUMode.SRIOV: 1>]
[16]


In [5]:
gpu = gpus[0]
print([x.fb_size/2**20 for x in gpu.supported_vgpu_types])

[5120.0, 10240.0, 20480.0, 20480.0, 40960.0, 4096.0, 5120.0, 8192.0, 10240.0, 20480.0, 40960.0, 5120.0]


In [51]:
gpu = gpus[0]
print(gpu.get_vgpu_max_instances(VGPUType.from_id(474)))
print(gpu.get_vf_available_vgpus(1))

7
[476: GRID A100-3-20C (MIG), 474: GRID A100-1-5C (MIG), 706: GRID A100-1-5CME (MIG)]


In [1]:
from pynvml import *
from pynvml import NVMLError_NotSupported
nvmlInit()

In [3]:
dev = nvmlDeviceGetHandleByIndex(0)
nvmlDeviceGetName(dev)

b'NVIDIA A100-PCIE-40GB'

In [4]:
info = nvmlDeviceGetPciInfo_v3(dev)

In [13]:
info.busIdLegacy

b'0000:41:00.0'

In [5]:
INVALID_GPU_INSTANCE_PROFILE_ID = 0xFFFFFFFF

gips = []
for i in range(NVML_GPU_INSTANCE_PROFILE_COUNT):
    try:
        gips.append(nvmlDeviceGetGpuInstanceProfileInfo(dev, i))
    except NVMLError_NotSupported:
        pass

for id in nvmlDeviceGetSupportedVgpus(dev):
    gi = nvmlVgpuTypeGetGpuInstanceProfileId(id)
    is_mig = gi != INVALID_GPU_INSTANCE_PROFILE_ID
    if is_mig:
        print(f"searching for GIP {gi}")
        print(gips)
        gi_info = next(x for x in gips if x.id == gi)

    print(f"{nvmlVgpuTypeGetName(id)} {'(MIG)' if is_mig else ''}")


NVMLError_NoPermission: Insufficient Permissions

In [ ]:
nvmlShutdown()